# Что делал с данными?




*   Брал данные с различных генераторов имен и фамили, чтобы были более разные имена
*   Приводил данные к общему виду, исключая повторы
*   Разделил данные по пробелу на имя и фамилию




In [1]:
import re

from collections import Counter

import numpy as np
import pandas as pd

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Dense, SimpleRNN, Input, LSTM, GRU

from sklearn.model_selection import train_test_split

# Создание датасета

## Датасет мужских имен

In [2]:
#from google.colab import files
 
 
#uploaded = files.upload()


In [4]:
data=pd.read_csv("https://raw.githubusercontent.com/Terkingil/Project_NLP/main/Models%20weights%20n%20data/Data%20for%20G%20model/Male%20Names.csv")# загружаем список имен
display(data)

,Name,Surname
0,Cezar,Rogov
1,Artur,Zhukov
2,Trofim,Zuev
3,Yan,Artamonov
4,Yurij,Lukashenko
...,...,...
295,NaN,Fomichyov
296,NaN,Zotov
297,NaN,Kulagin
298,NaN,Tokarev


Можно заметить что со 149 стоке столбцов имен стоят Nan 

In [5]:

Name_number=len(data)-data.loc[:,"Name"].isna().sum() #количество имен в таблице имен
Surname_number=len(data)

Name_array=np.array([])#массив с готовыми именами

for i in range(Name_number):#декартово произведение всех имен и фамилий чтобы получить различные имена и фамилии
   for j in range(Surname_number):
     tmp=data.iloc[i,0]+" "+data.iloc[j,1]
     Name_array=np.append(Name_array,tmp)



In [6]:
len(Name_array)

44700

In [7]:
M_Name=pd.DataFrame({"Name":Name_array,"Label":1})# Создаем датасет и присваиваем метку

In [8]:
M_Name

,Name,Label
0,Cezar Rogov,1
1,Cezar Zhukov,1
2,Cezar Zuev,1
3,Cezar Artamonov,1
4,Cezar Lukashenko,1
...,...,...
44695,Egor Fomichyov,1
44696,Egor Zotov,1
44697,Egor Kulagin,1
44698,Egor Tokarev,1


## Датасет женских имен

По сути, делаем тоже самое, что и на предыдущем  шаге

In [14]:
data=pd.read_csv("https://raw.githubusercontent.com/Terkingil/Project_NLP/main/Models%20weights%20n%20data/Data%20for%20G%20model/Female%20Names.csv")# загружаем список имен
display(data)

,Name,Surname
0,Mariya,Bobrova
1,Safiya,Vereshagina
2,Kamilla,Aleksandrova
3,Yana,Hudyakova
4,Alisa,Sinicyna
...,...,...
335,NaN,Ivashenko
336,NaN,Fokina
337,NaN,Tkachenko
338,NaN,Shufrich


In [15]:
Name_number=len(data)-data.loc[:,"Name"].isna().sum() #количество имен в таблице имен
Surname_number=len(data)

Name_array=np.array([])#массив с готовыми именами

for i in range(Name_number):#декартово произведение всех имен и фамилий чтобы получить различные имена и фамилии
   for j in range(Surname_number):
     tmp=data.iloc[i,0]+" "+data.iloc[j,1]
     Name_array=np.append(Name_array,tmp)

In [16]:
len(Name_array)

52700

In [17]:
F_Name=pd.DataFrame({"Name":Name_array,"Label":0})# Создаем датасет и присваиваем метку

In [18]:
F_Name

,Name,Label
0,Mariya Bobrova,0
1,Mariya Vereshagina,0
2,Mariya Aleksandrova,0
3,Mariya Hudyakova,0
4,Mariya Sinicyna,0
...,...,...
52695,Gera Ivashenko,0
52696,Gera Fokina,0
52697,Gera Tkachenko,0
52698,Gera Shufrich,0


## Соединяем  датасеты

In [19]:
Full_Name_df=M_Name.merge(F_Name,how='outer',on=["Name","Label"]) #Ползуюсь outer join здесь он наиболее подходит

In [20]:
Full_Name_df

,Name,Label
0,Cezar Rogov,1
1,Cezar Zhukov,1
2,Cezar Zuev,1
3,Cezar Artamonov,1
4,Cezar Lukashenko,1
...,...,...
97395,Gera Ivashenko,0
97396,Gera Fokina,0
97397,Gera Tkachenko,0
97398,Gera Shufrich,0


In [21]:
Full_Name_df.to_csv("Full_Names.csv",index=False) # - единый датасет с разными именами
#теперь на нем можно обучать модель

# Создание модели

In [22]:
cdt_Names=np.array(Full_Name_df.iloc[:,0])

print(cdt_Names[:5])

['Cezar Rogov' 'Cezar Zhukov' 'Cezar Zuev' 'Cezar Artamonov'
 'Cezar Lukashenko']


Токенизируем

In [23]:
#tokenizing Names
num_char = 28 # [a-z]+"-"+" "
tokenizer=Tokenizer(num_words=num_char, lower=True,char_level=True,filters="'!#$%&()*+,./:;<=>?@[\\]^_`{|}~\t\n'")
tokenizer.fit_on_texts(cdt_Names)
print(tokenizer.word_index)

inp_chars = 3
one_hot_enc=tokenizer.texts_to_sequences(cdt_Names)


{'a': 1, 'o': 2, 'v': 3, 'i': 4, 'n': 5, ' ': 6, 'e': 7, 'r': 8, 'l': 9, 's': 10, 'k': 11, 't': 12, 'y': 13, 'm': 14, 'h': 15, 'd': 16, 'u': 17, 'g': 18, 'b': 19, 'c': 20, 'z': 21, 'j': 22, 'p': 23, 'f': 24, '-': 25}


Паддинг, добавляем 0 в начало, т.к. так обычно работает быстрее обучение

In [24]:
print(cdt_Names[2])
print(one_hot_enc[2])


from keras_preprocessing.sequence import pad_sequences

train_pded=pad_sequences(one_hot_enc, padding="pre", truncating="post", maxlen=20) 

print(train_pded[2])

Cezar Zuev
[20, 7, 21, 1, 8, 6, 21, 17, 7, 3]
[ 0  0  0  0  0  0  0  0  0  0 20  7 21  1  8  6 21 17  7  3]


In [25]:
#samples



X=train_pded
y=np.array(Full_Name_df.iloc[:,1])


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=21)

X_train.shape,y_train.shape


((68180, 20), (68180,))

In [26]:
#model

model=Sequential()
model.add(Embedding(28,20,input_length=20))
model.add(GRU(500, activation='tanh',return_sequences=True))
model.add(GRU(250, activation='tanh'))
model.add(Dense(1, activation='sigmoid'))#Полносвязный слой
model.summary()

model.compile(loss='binary_crossentropy', metrics=["AUC"])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 20)            560       
                                                                 
 gru (GRU)                   (None, 20, 500)           783000    
                                                                 
 gru_1 (GRU)                 (None, 250)               564000    
                                                                 
 dense (Dense)               (None, 1)                 251       
                                                                 
Total params: 1,347,811
Trainable params: 1,347,811
Non-trainable params: 0
_________________________________________________________________


In [30]:
history=model.fit(X_train,y_train,batch_size=20,epochs=20,validation_data=(X_test,y_test))

In [ ]:
#model.save_weights('./G_model/GRU_weights_500_250')

# Проверка на ручном вводе

Здесь в квадратных скобках можно ввести имя, фамилию, имя фамилию и посмотреть что получится

0 - Ж \
1 - М


Если только имя, то после имени нужно поставить пробел
Если только фамилия, то никакими пробелами окружат её не надо

In [28]:
#model.load_weights('./G_model/GRU_weights_500_250')

In [29]:
inp=["Ivan Petrov"]

inp_enc=tokenizer.texts_to_sequences(inp)
#print(inp_enc)
inp_enc_pded=pad_sequences(inp_enc, padding="pre", truncating="post", maxlen=20)

#print(inp_enc_pded)

surname_hyppo = (model.predict(inp_enc_pded))

surname_hyppo

1/1 [==============================] - 1s 1s/step


array([[0.95927]], dtype=float32)